In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from nucleus.dataset.vq import VqDataset

In [ ]:
ds = VqDataset.from_s3(
    name='intermediate_jersey_dataset',
    bucket='hudlrd-experiments',
    key='jersey-tagging/development',
    n_jobs=None,
    show_progress=True
)

In [ ]:
ds.save(compress=True)

In [ ]:
ds2 = VqDataset.load(path='./dataset_cache/intermediate_jersey_dataset/dataset')

In [ ]:
ds2.view_row(0)

In [ ]:
image = ds2.images_lazy[0]

In [ ]:
x = []
for image in ds2.images_lazy:
    image.label

In [ ]:
for image in ds2.images_lazy[:2]:
    for img in image.images_from_box_collection(skip_labels=None):
        img.view(new_figure=True)

In [ ]:
import seaborn as sns



In [ ]:
type(ds.df.iloc[0])

In [ ]:
ds.images_lazy[0].box_collection.labels_list

In [ ]:
from nucleus.visualize.matplotlib import BasketballJerseyLabelColorMap

box_args = {
    'label_color_map': BasketballJerseyLabelColorMap,
    'alpha': 0.5,
    'caption_box_alpha': 0.5,
    'skip_labels': ['occluded', 'partial', 'unknown']
}

In [ ]:
image = ds.images_lazy[2]

In [ ]:
len(ds.images_lazy)

In [ ]:
image.view(box_args=box_args)